In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from __future__ import print_function, absolute_import, division

In [ ]:
import numpy as np
from wgomoku import GomokuBoard, GomokuTools as gt, NH9x9, Heuristics

In [ ]:
A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S,T,U = \
    1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
BLACK=0
WHITE=1
EDGES=2

## Single line scores

```Heuristics``` pre-computes all counts and scores a initialization time 

In [ ]:
h = Heuristics(kappa=3.0)

Displaying the relevance count, which is a function of the number of offensive stones, their density, and open-ended-ness. Also displaying the score which adds some more fine-grained heuristic relevance aspects.

In [ ]:
gt.line_for_xo('...xxo..')

In [ ]:
h.line_score_for(24,4)

In [ ]:
h.lookup_line_score(24,4)

In [ ]:
h.lookup_total_scores(np.array([[[4, 5, 0, 0],[4, 7.5, 0, 0]]]))

In [ ]:
for i in [
    '...ox...',
    '...x..xo',
    '.....x..',
    '...x.xo.',
    '...x..x.',
    '...xxo..',
    'xx......',
    '...x.x..',
    '..oxx.x.',
    '...xx...',
    '.xx.....',
    '.....xxx',
    '...xxxo.',
    '...xx.x.',
    '...xxx..',
    '.oxxxx..',
    '..xxxx..']:
    print(i, h.lookup_line_score(*gt.line_for_xo(i)))

## Multi-line (neighbourhood) scores

In [ ]:
line_counts=9-(np.sqrt(np.random.randint(0,18**2, [5, 5, 4]))).astype(int)/2

In [ ]:
line_counts

In [ ]:
h.lookup_total_scores(line_counts)

---
Some tools for visualization

In [ ]:
def nhforxo(e,ne='........', n='........', nw='........'):
    """
    Neighbourhood from xo-strings
    """
    lines = np.array([gt.line_for_xo(xo) for xo in [e,ne,n,nw]])
    lines = np.rollaxis(lines, 1, 0)
    return NH9x9(*lines)

In [ ]:
def dsc(h, e,ne='........',n='........', nw='........', kappa=1.2):
    """
    Display the neighbourhood and its score
    """
    counts = [h.lookup_line_score(*gt.line_for_xo(s)) for s in [e,ne,n,nw]]
    print(counts)
    print(h.lookup_total_scores(np.array([[counts]])))
    print(nhforxo(e,ne,n,nw))

---

#### Not strictly dangerous, yet.

In [ ]:
dsc(h, '.....xxx')

In [ ]:
dsc(h, '.....xxx', '....xx..')

In [ ]:
dsc(h, '...xx..o', '...x.xo.')

In [ ]:
dsc(h, '...xxxo.','..x.....')

### Soft attack

In [ ]:
dsc(h, '...x.x.o', '...xx...')

In [ ]:
dsc(h, '...xx..o', '...xx...')

#### Hard attack

In [ ]:
dsc(h, '...xxxo.','.xx.....')

#### Fatal

In [ ]:
dsc(h, '...xxx.o')

#### Dead

In [ ]:
dsc(h, '...xxxxo')

### Considering all options for critical defense states
The defense of critical positions may not only come through occupation of that position but also through any other position within the 9x9 neighbourhood of that position. Thus, when there is a critical defense situation, all candidate positions must be checked by test-occupying them and checking whether the originally alerting position's criticality is alleviated by that stone. The policy should meaningfully choose between all those potentially alleviating options. Not too smartly though, for that's what we'll have the Deep Q-Network for. 

### Determining an estimate for the Q-Values
#### An estimate for the state's value
The difference between the sums of offensive scores and defensive scores may be a good first estimate for the value of a board state. 


#### Q-Value Estimation for non-critical states
We could use the top10 of each style -> 30 positions, occupy each, calculate the state value afterwards to obtain s,a->v triples. All other s,a combinations are considered bad, and may get a penalty as the "value of an irrelevant move followed by opponent's best response", subtracted from the board's current value